In [1]:
%run Weighting.ipynb

Pixiedust database opened successfully


In [2]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - UNDIP (IPA),Agroekoteknologi - UB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),Antropologi - UNAIR (IPS),...,Teknik Sistem Perkapalan - ITS (IPA),Teknobiomedik - UNAIR (IPA),Teknologi Bioproses - UI (IPA),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - UGM (IPA),Teknologi Pangan - UB (IPA),Televisi & Film - UNPAD (IPS),Tip - UGM (IPA),Tv & Film - UNPAD (IPS)
Matematika,84.000000,88.500000,85.465625,92.000000,86.250000,81.852143,82.500000,91.625000,44.693125,84.576004,...,90.000000,86.154669,85.465625,82.000000,85.000000,86.506173,82.408571,85.000000,87.000000,85.000000
Inggris,84.000000,80.500000,83.888667,86.000000,85.500000,84.729722,82.750000,90.875000,54.255467,85.307966,...,87.000000,86.153983,83.888667,91.000000,84.000000,86.859259,83.918889,85.500000,94.000000,85.000000
Indonesia,84.000000,89.500000,85.246032,89.000000,87.833333,85.837917,84.000000,89.875000,85.000000,85.970128,...,88.000000,86.651731,85.246032,89.000000,86.000000,86.982716,84.351667,86.500000,80.000000,84.000000
Fisika,82.000000,84.584430,84.493016,81.956615,83.360000,81.739154,80.500000,89.437576,87.000000,84.646720,...,88.500000,85.048360,84.493016,80.000000,83.000000,85.562309,81.956615,84.584430,86.000000,84.584430
Kimia,81.000000,84.592411,85.297674,82.397782,85.100000,82.349446,81.500000,88.716106,74.300000,85.035214,...,89.500000,85.286546,85.297674,79.000000,81.000000,86.895425,82.397782,84.592411,87.000000,84.592411
Biologi,83.000000,83.879942,84.182558,82.827536,86.172727,83.706884,83.250000,89.695798,83.000000,86.206897,...,92.000000,86.364812,84.182558,83.000000,86.000000,87.039216,82.827536,83.879942,86.000000,83.879942
Ekonomi,86.145714,89.000000,86.145714,91.242143,87.271429,83.726429,82.062045,92.250000,86.145714,85.211111,...,87.839744,85.654365,86.145714,88.726429,85.211111,87.864219,84.210714,86.000000,89.837895,86.000000
Geografi,85.512727,87.500000,85.512727,91.000000,87.500000,84.627747,81.926031,88.959803,85.512727,84.793706,...,84.333333,85.363520,85.512727,84.808608,84.793706,88.470835,84.808608,86.500000,88.094322,87.000000
Sosiologi,86.930877,85.500000,86.930877,91.000000,87.000000,85.702500,81.331846,91.872475,86.930877,85.695971,...,84.666667,85.531319,86.930877,84.850000,85.695971,91.380471,84.850000,85.500000,90.363636,85.000000


In [3]:
nilai_jams = [70, 70, 70, 70, 70, 70, 70, 70, 70,]

In [4]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

[-2.749822364309754,
 -3.638069364917934,
 -4.8201766621884445,
 -3.7731279264212465,
 -3.605954356872745,
 -4.615593197475401,
 -6.099829348527156,
 -6.445788711418059,
 -5.758986068783327]

In [5]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - UNDIP (IPA),Agroekoteknologi - UB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),Antropologi - UNAIR (IPS),...,Teknobiomedik - UNAIR (IPA),Teknologi Bioproses - UI (IPA),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - UGM (IPA),Teknologi Pangan - UB (IPA),Televisi & Film - UNPAD (IPS),Tip - UGM (IPA),Tv & Film - UNPAD (IPS),Parameter
Matematika,84.000000,88.500000,85.465625,92.000000,86.250000,81.852143,82.500000,91.625000,44.693125,84.576004,...,86.154669,85.465625,82.000000,85.000000,86.506173,82.408571,85.000000,87.000000,85.000000,-2.749822
Inggris,84.000000,80.500000,83.888667,86.000000,85.500000,84.729722,82.750000,90.875000,54.255467,85.307966,...,86.153983,83.888667,91.000000,84.000000,86.859259,83.918889,85.500000,94.000000,85.000000,-3.638069
Indonesia,84.000000,89.500000,85.246032,89.000000,87.833333,85.837917,84.000000,89.875000,85.000000,85.970128,...,86.651731,85.246032,89.000000,86.000000,86.982716,84.351667,86.500000,80.000000,84.000000,-4.820177
Fisika,82.000000,84.584430,84.493016,81.956615,83.360000,81.739154,80.500000,89.437576,87.000000,84.646720,...,85.048360,84.493016,80.000000,83.000000,85.562309,81.956615,84.584430,86.000000,84.584430,-3.773128
Kimia,81.000000,84.592411,85.297674,82.397782,85.100000,82.349446,81.500000,88.716106,74.300000,85.035214,...,85.286546,85.297674,79.000000,81.000000,86.895425,82.397782,84.592411,87.000000,84.592411,-3.605954
Biologi,83.000000,83.879942,84.182558,82.827536,86.172727,83.706884,83.250000,89.695798,83.000000,86.206897,...,86.364812,84.182558,83.000000,86.000000,87.039216,82.827536,83.879942,86.000000,83.879942,-4.615593
Ekonomi,86.145714,89.000000,86.145714,91.242143,87.271429,83.726429,82.062045,92.250000,86.145714,85.211111,...,85.654365,86.145714,88.726429,85.211111,87.864219,84.210714,86.000000,89.837895,86.000000,-6.099829
Geografi,85.512727,87.500000,85.512727,91.000000,87.500000,84.627747,81.926031,88.959803,85.512727,84.793706,...,85.363520,85.512727,84.808608,84.793706,88.470835,84.808608,86.500000,88.094322,87.000000,-6.445789
Sosiologi,86.930877,85.500000,86.930877,91.000000,87.000000,85.702500,81.331846,91.872475,86.930877,85.695971,...,85.531319,86.930877,84.850000,85.695971,91.380471,84.850000,85.500000,90.363636,85.000000,-5.758986


In [6]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

for pilihan in pilihan_jurusan:
    if "(ipa)" in pilihan.casefold().split():
        print(pilihan)

Agribisnis - UNDIP (IPA)
Agroekoteknologi - UB (IPA)
Agroteknologi - UNPAD (IPA)
Apoteker - UNAIR (IPA)
Arsitektur - ITS (IPA)
Astronomi - ITB (IPA)
Biologi - ITS (IPA)
Budidaya Perairan - UNAIR (IPA)
Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)
Fakultas Kedokteran Gigi - UNAIR (IPA)
Fakultas Teknik Sipil & Lingkungan - ITB (IPA)
Farmasi - UI (IPA)
Fisika - ITS (IPA)
Fitb - ITB (IPA)
Fkg - UGM (IPA)
Fmipa - ITB (IPA)
Fmipa Fisika - UB (IPA)
Fmipa- Ilmu Kimia - UNPAD (IPA)
Fti - ITB (IPA)
Geofisika - UGM (IPA)
Geofisika  - UI (IPA)
Gizi - UGM (IPA)
Ilmu Gizi - UI (IPA)
Ilmu Hub. Internasional - UNPAD (IPA)
Ilmu Kelautan - UNDIP (IPA)
Ilmu Keperawatan - UI (IPA)
Ilmu Keperawatan  - UNPAD  (IPA)
Ilmu Kesehatan Masyarakat - UI (IPA)
Ilmu Komputer - UGM (IPA)
Ilmu Peternakan - UNPAD (IPA)
Itb - Sithr - ITB (IPA)
Kedokteran - UI (IPA)
Kedokteran  - UNAIR (IPA)
Kedokteran Gigi - UGM (IPA)
Kedokteran Hewan - UNAIR (IPA)
Kehutanan - UGM (IPA)
Kesehatan Masyarakat - UI (IPA)
Kesehatan Masyar

In [7]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - UNDIP (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPS),Administrasi Fiskal - UI (IPS)&Antropologi Sosial - UB (IPS),...,Tv & Film - UNPAD (IPS)&Teknik Sipil - ITB (IPA),Tv & Film - UNPAD (IPS)&Teknik Sistem Perkapalan - ITS (IPA),Tv & Film - UNPAD (IPS)&Teknobiomedik - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - UB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Matematika,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0,0.0,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
Inggris,0.000000,0.055667,-1.0,-0.750000,-0.364861,0.625,0,0.0,-0.653983,0,...,0.869444,-1.000000,-0.576991,0.555667,0.000,0.500000,-0.929630,0.540556,-0.25,0.000000
Indonesia,0.000000,-0.623016,0.0,0.000000,-0.918958,0.000,0,-0.5,-0.985064,0,...,0.908046,0.000000,0.000000,-0.623016,0.000,-1.000000,0.000000,-0.175833,0.00,0.000000
Fisika,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0,0.0,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
Kimia,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0,0.0,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
Biologi,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0,0.0,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
Ekonomi,0.000000,0.000000,0.0,-0.562857,0.000000,0.000,0,0.0,0.467302,0,...,-0.958712,-0.919872,0.172817,-0.072857,0.000,0.394444,-0.932109,0.894643,0.00,0.000000
Geografi,-0.993636,0.000000,0.0,-0.993636,0.442490,0.000,0,0.0,0.359510,0,...,0.570536,0.000000,0.818240,0.743636,0.000,0.000000,-0.735417,0.000000,0.25,-0.547161
Sosiologi,0.715439,0.000000,0.0,-0.034561,0.614189,0.000,0,0.0,0.617453,0,...,0.000000,0.166667,-0.265659,-0.965439,0.075,-0.347985,0.000000,0.075000,-0.25,0.000000


In [8]:
index_preferensi_global = idx_pref_global(fp_gaussian, weight)

index_preferensi_global

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - UNDIP (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPS),Administrasi Fiskal - UI (IPS)&Antropologi Sosial - UB (IPS),...,Tv & Film - UNPAD (IPS)&Teknik Sipil - ITB (IPA),Tv & Film - UNPAD (IPS)&Teknik Sistem Perkapalan - ITS (IPA),Tv & Film - UNPAD (IPS)&Teknobiomedik - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - UB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Index Preferensi Global,-0.029318,-0.059789,-0.105384,-0.246709,-0.023937,0.065865,0.0,-0.052692,-0.020527,0.0,...,0.146411,-0.184759,0.01564,-0.03815,0.007904,-0.047796,-0.273698,0.140621,-0.026346,-0.057662


In [9]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

e_flow

{'Administrasi Fiskal - UI (IPS)': 0.030883752677037732,
 'Administrasi Negara - UNPAD (IPS)': -0.013161398671080096,
 'Administrasi Niaga - UI (IPS)': 0.011413830218463854,
 'Administrasi Perpajakan - UB (IPS)': -0.008740064326422619,
 'Agribisnis - UNDIP (IPA)': -0.012850216645272226,
 'Agroekoteknologi - UB (IPA)': 0.008496930085538026,
 'Agroteknologi - UNPAD (IPA)': 0.03938936340611822,
 'Akuntansi - UGM (IPS)': -0.041995421065397734,
 'Akutansi - UI (IPS)': 0.005259639521216939,
 'Antropologi - UNAIR (IPS)': 0.003965816550415996,
 'Antropologi Sosial - UB (IPS)': 0.015204067129142386,
 'Apoteker - UNAIR (IPA)': -0.024669162955880848,
 'Arkeologi - UI (IPS)': -0.006575744235125748,
 'Arsitektur - ITS (IPA)': 0.008345178811666451,
 'Astronomi - ITB (IPA)': 0.012820525992401145,
 'Biologi - ITS (IPA)': 0.03079110529407458,
 'Budidaya Perairan - UNAIR (IPA)': 0.006980293160464597,
 'Desain Interior - ITS (IPS)': -0.014518946574345246,
 'Ekonomi Islam - UB (IPS)': 0.035115813820556896

In [10]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

l_flow

{'Administrasi Fiskal - UI (IPS)': -0.01119478758360488,
 'Administrasi Negara - UNPAD (IPS)': 0.040205736592996706,
 'Administrasi Niaga - UI (IPS)': 0.005024625927259107,
 'Administrasi Perpajakan - UB (IPS)': 0.01996841527095168,
 'Agribisnis - UNDIP (IPA)': 0.011777667918366902,
 'Agroekoteknologi - UB (IPA)': -0.005620521003457821,
 'Agroteknologi - UNPAD (IPA)': -0.051878417834434566,
 'Akuntansi - UGM (IPS)': 0.04346855224752557,
 'Akutansi - UI (IPS)': 0.0069791856296206765,
 'Antropologi - UNAIR (IPS)': 0.002759323875591457,
 'Antropologi Sosial - UB (IPS)': -0.030416514408986777,
 'Apoteker - UNAIR (IPA)': 0.0255663260354271,
 'Arkeologi - UI (IPS)': 0.044350088371420814,
 'Arsitektur - ITS (IPA)': -0.01377077735789953,
 'Astronomi - ITB (IPA)': -0.019423086904172063,
 'Biologi - ITS (IPA)': -0.03319770311188558,
 'Budidaya Perairan - UNAIR (IPA)': -0.00959698551886639,
 'Desain Interior - ITS (IPS)': -0.0012711675522712829,
 'Ekonomi Islam - UB (IPS)': -0.05282371696514046,


In [11]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

{'Administrasi Fiskal - UI (IPS)': -0.04207854026064261,
 'Administrasi Negara - UNPAD (IPS)': 0.0533671352640768,
 'Administrasi Niaga - UI (IPS)': -0.006389204291204748,
 'Administrasi Perpajakan - UB (IPS)': 0.028708479597374298,
 'Agribisnis - UNDIP (IPA)': 0.024627884563639128,
 'Agroekoteknologi - UB (IPA)': -0.014117451088995847,
 'Agroteknologi - UNPAD (IPA)': -0.09126778124055279,
 'Akuntansi - UGM (IPS)': 0.0854639733129233,
 'Akutansi - UI (IPS)': 0.0017195461084037371,
 'Antropologi - UNAIR (IPS)': -0.0012064926748245391,
 'Antropologi Sosial - UB (IPS)': -0.04562058153812916,
 'Apoteker - UNAIR (IPA)': 0.05023548899130795,
 'Arkeologi - UI (IPS)': 0.05092583260654656,
 'Arsitektur - ITS (IPA)': -0.022115956169565982,
 'Astronomi - ITB (IPA)': -0.03224361289657321,
 'Biologi - ITS (IPA)': -0.06398880840596016,
 'Budidaya Perairan - UNAIR (IPA)': -0.016577278679330988,
 'Desain Interior - ITS (IPS)': 0.013247779022073964,
 'Ekonomi Islam - UB (IPS)': -0.08793953078569736,
 '

In [12]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

tabel_hasil

,Jurusan,Net Flow
1,Fkg - UGM (IPA),0.089015
2,Akuntansi - UGM (IPS),0.085464
3,Stei - ITB (IPA),0.082239
4,Teknik Industri - ITB (IPA),0.072641
5,Psdk - UGM (IPS),0.067358
...,...,...
133,Ilmu Kelautan - UNDIP (IPA),-0.088693
134,Agroteknologi - UNPAD (IPA),-0.091268
135,Ekonomi Islam - UNPAD (IPS),-0.091849
136,Ilmu Hukum - UB (IPS),-0.113080
